In [ ]:
import threading
import time
import socket
from _thread import *
from Oracle_ML_Model import MyModel
import pandas as pd
import re

print_lock = threading.Lock()

class OracleBotThread(threading.Thread):
    def __init__(self, client_socket, client_address):
        threading.Thread.__init__(self)
        self.client_socket = client_socket
        self.client_address = client_address
        # Initialize ML model here
        self.my_model = None  # Model will be initialized after getting the date from the client
        
    def run(self):
        self.greet_user()  # Greet the user first
        self.ask_for_current_date()  # Then ask for the current date
        self.handle_conversation()  # Handle the rest of the conversation
        self.goodbye_user()  # Say goodbye when the conversation ends

    def greet_user(self):
        self.send_message("Hello, I'm Oracle, your weather and bike hire prediction assistant. "
                          "To provide accurate predictions, I need to know today's date. "
                          "Please enter the date in YYYY-MM-DD format.")
        time.sleep(1)
    
    def ask_for_current_date(self):
        while True:
            date_str = self.receive_message().strip()
            try:
                current_date = pd.to_datetime(date_str)
                self.my_model = MyModel(current_date)  # Initialize the ML model with the date
                self.send_message(f"Thank you. I have set the date to {current_date.strftime('%Y-%m-%d')}. "
                                  "You can now ask me about the weather or bike hire numbers for the next 7 days. "
                                  "For example, 'What's the weather like on day 3?' or 'Predict bike hires for day 5.'")
                break  # Exit the loop if the date is valid
            except ValueError:
                self.send_message("The date format is incorrect. Please enter the date in YYYY-MM-DD format.")

    def handle_conversation(self):
        try:
            while True:
                user_input = self.receive_message().lower()
    
                day = self.extract_day(user_input)
    
                if "weather" in user_input:
                    self.predict_weather(day)
                elif "bike hires" in user_input:
                    self.predict_bike_hires(day)
                elif "goodbye" in user_input:
                    break
                else:
                    self.send_message("I'm sorry, I didn't understand. Ask about weather or bike hires.")
        except Exception as e:
            print(f"Error in handle_conversation: {e}")
    
    def extract_day(self, user_input):
        # Use expressions to find day numbers in the message
        match = re.search(r'\bday\s(\d+)', user_input)
        if match:
            day = int(match.group(1))
            if 1 <= day <= 7:
                return day
        return 1  # Default to Day 1 if no valid day is found

    def goodbye_user(self):
        self.send_message("Thank you for consulting the Oracle. Have a great day!")
        time.sleep(1)
        self.client_socket.close()
        print_lock.release()

    def predict_weather(self, day):
        date_str = self.day_to_date(day)
        weather_features = ['tempmax', 'tempmin', 'windspeed', 'precip']
        predictions = [self.my_model.predict(day, feature) for feature in weather_features]
    
        response = (f"For {date_str}, the forecast is: "
                    f"high of {predictions[0]:.1f}°C, low of {predictions[1]:.1f}°C, "
                    f"wind speed at {predictions[2]:.1f} km/h, and precipitation levels around {predictions[3]:.1f} mm.")
        self.send_message(response)
    
    def predict_bike_hires(self, day):
        date_str = self.day_to_date(day)
        value = self.my_model.predict(day, 'Number of Bicycle Hires')
    
        response = f"For {date_str}, the expected number of bike hires is approximately {value:.0f}."
        self.send_message(response)

    def send_message(self, message):
        try:
            print("Sending message:", message)
            self.client_socket.send(message.encode('utf-8'))
        except Exception as e:
            print(f"Error in send_message: {e}")

    def receive_message(self):
        try:
            data = self.client_socket.recv(1024)
            user_input = data.decode('utf-8')
            print("Received message:", user_input)
            return user_input
        except Exception as e:
            print(f"Error in receive_message: {e}")
            return ""

    def day_to_date(self, day):
            # Convert the day number to a date string
            return f"Day {day}"

def Main():
    host = "127.0.0.1"
    port = 12345

    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind((host, port))
    print("Socket bound to port", port)

    s.listen(5)
    print("Socket is listening")

    while True:
        c, addr = s.accept()    # Establish connection with the client
        print_lock.acquire()    # Lock acquired by the client
        print("Connected to:", addr[0], ":", addr[1])

        oracle_thread = OracleBotThread(c, addr)
        oracle_thread.start()

if __name__ == "__main__":
    Main()

Socket bound to port 12345
Socket is listening
Connected to: 127.0.0.1 : 54714
Sending message: Hello, I'm Oracle, your weather and bike hire prediction assistant. To provide accurate predictions, I need to know today's date. Please enter the date in YYYY-MM-DD format.
Received message: 2023-08-32
Sending message: The date format is incorrect. Please enter the date in YYYY-MM-DD format.
Received message: 2023-08-22


C:\Users\940167\SNS Final\Oracle_ML_Model.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset1_filtered.drop(columns=['name', 'severerisk', 'windgust',


Sending message: Thank you. I have set the date to 2023-08-22. You can now ask me about the weather or bike hire numbers for the next 7 days. For example, 'What's the weather like on day 3?' or 'Predict bike hires for day 5.'
Received message: how is the weather forecast on day 6?
1/1 [==============================] - 0s 95ms/step
Sending message: For Day 6, the forecast is: high of 6.4°C, low of 5.5°C, wind speed at -0.5 km/h, and precipitation levels around -1.7 mm.
Received message: what is the estimation for number of bike hires on day 7?
1/1 [==============================] - 2s 2s/step
Sending message: For Day 7, the expected number of bike hires is approximately 1447.
Received message: goodbye
Sending message: Thank you for consulting the Oracle. Have a great day!
Connected to: 127.0.0.1 : 54717
Sending message: Hello, I'm Oracle, your weather and bike hire prediction assistant. To provide accurate predictions, I need to know today's date. Please enter the date in YYYY-MM-DD fo

C:\Users\940167\SNS Final\Oracle_ML_Model.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset1_filtered.drop(columns=['name', 'severerisk', 'windgust',


Sending message: Thank you. I have set the date to 2024-01-02. You can now ask me about the weather or bike hire numbers for the next 7 days. For example, 'What's the weather like on day 3?' or 'Predict bike hires for day 5.'
Received message: what is the weather like on day 3?
1/1 [==============================] - 0s 98ms/step
Sending message: For Day 3, the forecast is: high of -5.0°C, low of -1.9°C, wind speed at 6.2 km/h, and precipitation levels around -1.1 mm.
Received message: how about the number of bike hires on day 3?
1/1 [==============================] - 0s 128ms/step
Sending message: For Day 3, the expected number of bike hires is approximately -8285.
Received message: thank you
Sending message: I'm sorry, I didn't understand. Ask about weather or bike hires.
Received message: goodbye
Sending message: Thank you for consulting the Oracle. Have a great day!
